In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import math
import string as str

In [79]:
question_df = pd.read_csv("america_in_one_room/Stanford_A1R_Codebook B.csv")

In [80]:
questions_extracted = question_df[question_df['Variable'].notnull()]
questions_extracted = questions_extracted[questions_extracted['Variable'].str.startswith("Q")].reset_index()
questions_extracted.iloc[0:48][['Variable','Variable Label']].to_csv("just_questions")

In [81]:
study_df = pd.read_csv("america_in_one_room/Stanford_A1R_Dataset_APSR.csv")


In [82]:
post_opinion_df = study_df[study_df['POST'] == 1]

In [83]:
grouped = post_opinion_df[post_opinion_df['GROUP'] != " "]
control = post_opinion_df[post_opinion_df['GROUP'] == " "]

In [84]:
race_dict = {1:"White, non-Hispanic", 2:"Black, non-Hispanic", 3:"Other, non-Hispanic", 4:"Hispanic", 5:"2+, non-Hispanic", 6:"Asian, non-Hispanic"}
education_dict = {1:"No HS diploma", 2:"HS graduate or equivalent", 3:"Some college", 4:"BA or above"}
gender_dict = {0:"Unknown", 1:"Male", 2:"Female"}
age_categories_dict = {1:"18-29", 2:"30-44", 3:"45-59", 4:"60+"}
party_id = {1:"Democrat", 2:"Republican", 3:"Independent"}

In [85]:
grouped["RACE"] = grouped.apply(lambda x: race_dict[x['RACETHNICITY']], axis= 1)
grouped["AGEBRACKET"] = grouped.apply(lambda x: age_categories_dict[x['AGE4']], axis= 1)
grouped["EDUCATION"] = grouped.apply(lambda x: education_dict[x['EDUC4']], axis= 1)
grouped["GENDER"] = grouped.apply(lambda x: gender_dict[x['GENDER']], axis= 1)
grouped["PARTYBEFORE"] = grouped.apply(lambda x: party_id[x['PARTYID3']], axis= 1)
#grouped["PARTYAFTER"] = grouped.apply(lambda x: party_id[int(x['T2PARTYID3'])], axis= 1)

/var/folders/qw/dj7vjbyd6kb8q83q6chjb9m00000gn/T/ipykernel_16140/3404512867.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped["RACE"] = grouped.apply(lambda x: race_dict[x['RACETHNICITY']], axis= 1)
/var/folders/qw/dj7vjbyd6kb8q83q6chjb9m00000gn/T/ipykernel_16140/3404512867.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped["AGEBRACKET"] = grouped.apply(lambda x: age_categories_dict[x['AGE4']], axis= 1)
/var/folders/qw/dj7vjbyd6kb8q83q6chjb9m00000gn/T/ipykernel_16140/3404512867.py:3: Sett

In [86]:
grouped.head()

,WEIGHT_CONTROL,WEIGHT_DELEGATE,POST,GROUP,CONDITION,Q1,Q2A,Q2B,Q2C,Q2D,...,T2PARTYID3,GENDER,AGE,AGE4,RACETHNICITY,EDUC4,RACE,AGEBRACKET,EDUCATION,PARTYBEFORE
0,,0.712025789,1,1,1,5,0,10,0,5,...,3,Male,21,1,1,3,"White, non-Hispanic",18-29,Some college,Independent
6,,0.918515451,1,24,1,7,10,8,0,0,...,2,Male,34,2,1,4,"White, non-Hispanic",30-44,BA or above,Republican
12,,0.51365971,1,9,1,7,0,10,10,6,...,1,Female,71,4,2,4,"Black, non-Hispanic",60+,BA or above,Democrat
14,,1.465232705,1,25,1,1,0,10,10,10,...,1,Female,77,4,1,4,"White, non-Hispanic",60+,BA or above,Democrat
20,,0.727367528,1,21,1,5,10,10,5,5,...,2,Male,51,3,1,3,"White, non-Hispanic",45-59,Some college,Republican


In [93]:
clean_df = grouped[['GROUP', 'CONDITION', 'Q1', 'Q2A', 'Q2B', 'Q2C', 'Q2D', 'Q2E', 'Q2F', 'Q2G', 'Q2H', 'Q2I', 'Q3A', 'Q3B', 'Q3C', 'Q3D', 'Q3E', 'Q3F', 'Q3G', 'Q3H', 'Q4A', 'Q4B', 'Q4C', 'Q4D', 'Q4E', 'Q4F', 'Q4G', 'Q4H', 'Q4I', 'Q4J', 'Q5A', 'Q5B', 'Q5C', 'Q5D', 'Q5E', 'Q5F', 'Q5G', 'Q5H', 'Q5I', 'Q5J', 'Q6A', 'Q6B', 'Q6C', 'Q6D', 'Q6E', 'Q6F', 'Q6G', 'Q6H', 'Q6I', 'Q6J', 'T2Q1', 'T2Q2A', 'T2Q2B', 'T2Q2C', 'T2Q2D', 'T2Q2E', 'T2Q2F', 'T2Q2G', 'T2Q2H', 'T2Q2I', 'T2Q3A', 'T2Q3B', 'T2Q3C', 'T2Q3D', 'T2Q3E', 'T2Q3F', 'T2Q3G', 'T2Q3H', 'T2Q4A', 'T2Q4B', 'T2Q4C', 'T2Q4D', 'T2Q4E', 'T2Q4F', 'T2Q4G', 'T2Q4H', 'T2Q4I', 'T2Q4J', 'T2Q5A', 'T2Q5B', 'T2Q5C', 'T2Q5D', 'T2Q5E', 'T2Q5F', 'T2Q5G', 'T2Q5H', 'T2Q5I', 'T2Q5J', 'T2Q6A', 'T2Q6B', 'T2Q6C', 'T2Q6D', 'T2Q6E', 'T2Q6F', 'T2Q6G', 'T2Q6H', 'T2Q6I', 'RACE', 'AGEBRACKET', 'EDUCATION', 'PARTYBEFORE', 'GENDER', 'AGE']].reset_index()

In [94]:
clean_df

,index,GROUP,CONDITION,Q1,Q2A,Q2B,Q2C,Q2D,Q2E,Q2F,...,T2Q6F,T2Q6G,T2Q6H,T2Q6I,RACE,AGEBRACKET,EDUCATION,PARTYBEFORE,GENDER,AGE
0,0,1,1,5,0,10,0,5,5,5,...,9,5,6,0,"White, non-Hispanic",18-29,Some college,Independent,Male,21
1,6,24,1,7,10,8,0,0,0,10,...,0,0,6,10,"White, non-Hispanic",30-44,BA or above,Republican,Male,34
2,12,9,1,7,0,10,10,6,10,8,...,10,8,6,5,"Black, non-Hispanic",60+,BA or above,Democrat,Female,71
3,14,25,1,1,0,10,10,10,10,7,...,10,10,6,7,"White, non-Hispanic",60+,BA or above,Democrat,Female,77
4,20,21,1,5,10,10,5,5,5,10,...,6,2,5,3,"White, non-Hispanic",45-59,Some college,Republican,Male,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,3803,32,1,5,5,7,2,5,10,10,...,6,10,5,5,"White, non-Hispanic",45-59,HS graduate or equivalent,Republican,Female,58
519,3815,16,1,3,4,10,10,8,10,9,...,5,10,8,1,"White, non-Hispanic",30-44,Some college,Independent,Female,31
520,3820,13,1,8,3,10,10,10,10,10,...,7,9,6,5,"White, non-Hispanic",60+,Some college,Independent,Female,63
521,3836,13,1,5,4,10,4,7,6,10,...,10,5,7,0,"White, non-Hispanic",45-59,BA or above,Democrat,Male,47
